# Features de TextMining:

---------------------------------

In [135]:
import os
import sys
import re
import datetime
import dateutil

sys.path.insert(0,os.path.dirname(os.getcwd()))
sys.path.insert(0,os.path.join(os.getcwd(),'grobid'))
sys.path.insert(0,os.getcwd())

import plotly
import numpy as np
import pandas as pd

In [136]:
import nltk

In [137]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\danie\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [138]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\danie\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [139]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\danie\AppData\Roaming\nltk_data...


True

- Countries data:

In [98]:
df_cont = pd.read_csv('../data/0_external/countries.csv',sep=',')

col_utils = ['name','region','sub-region','alpha-2','alpha-3']
df_cont = df_cont.loc[:,col_utils].copy()

df_cont.rename(columns={'name':'country'}, inplace=True)

df_cont.head()

,country,region,sub-region,alpha-2,alpha-3
0,Afghanistan,Asia,Southern Asia,AF,AFG
1,Åland Islands,Europe,Northern Europe,AX,ALA
2,Albania,Europe,Southern Europe,AL,ALB
3,Algeria,Africa,Northern Africa,DZ,DZA
4,American Samoa,Oceania,Polynesia,AS,ASM


In [116]:
df_sample = df_cont.sample(frac=2, replace=True)
df_sample = df_sample.fillna('Unidentified')

- Agg data:

In [117]:
df_agg = df_sample.groupby(by=['region','sub-region','country'], as_index=False)['alpha-2'].count()
df_agg_region = df_sample.groupby(by=['region'], as_index=False)['alpha-2'].count()
df_agg_subregion = df_sample.groupby(by=['region','sub-region'], as_index=False)['alpha-2'].count()

- Link information:

In [119]:
link_region = df_agg_subregion.apply(lambda line: {'source':line['region'],
                                     'target':line['sub-region'],
                                     'value':line['alpha-2']}, axis=1).tolist()

link_sub_region = df_agg.apply(lambda line: {'source':line['sub-region'],
                                             'target':line['country'],
                                             'value':line['alpha-2']}, axis=1).tolist()

In [120]:
df_link_region = pd.DataFrame(link_region).sample(10)
df_link_sub_region = pd.DataFrame(link_sub_region).sample(10)
df_links = pd.concat([df_link_region, df_link_sub_region])

- Node information:

In [121]:
nodes = list(df_links.source.unique()) + list(df_links.target.unique())
nodes = list(np.unique(nodes))
dict_code_nodes = {e:i for i,e in enumerate(nodes, start=0)}
dict_code_nodes

{'Americas': 0,
 'Asia': 1,
 'Australia and New Zealand': 2,
 'Central Asia': 3,
 'Eastern Asia': 4,
 'Eastern Europe': 5,
 'Europe': 6,
 'French Southern Territories': 7,
 'Hong Kong': 8,
 'Kuwait': 9,
 'Latin America and the Caribbean': 10,
 'Libya': 11,
 'Lithuania': 12,
 'Melanesia': 13,
 'Northern Africa': 14,
 'Northern America': 15,
 'Northern Europe': 16,
 'Oceania': 17,
 'Sint Maarten (Dutch part)': 18,
 'South-eastern Asia': 19,
 'Southern Asia': 20,
 'Sub-Saharan Africa': 21,
 'Turkmenistan': 22,
 'Uganda': 23,
 'Unidentified': 24,
 'Uzbekistan': 25,
 'Western Asia': 26,
 'Western Europe': 27,
 'Zambia': 28}

In [122]:
f_rand_color = lambda e: f'rgba({np.random.randint(0,255)},{np.random.randint(0,255)},{np.random.randint(0,255)}, 0.8)'

colors_nodes = [ f_rand_color(e) for e in dict_code_nodes.keys()]

dict_colors_nodes = { node:color for node, color in zip(nodes,colors_nodes)}

In [123]:
df_links['label'] = df_links['target']
df_links['source'] = df_links['source'].apply(lambda e: dict_code_nodes.get(e,-1))
df_links['target'] = df_links['target'].apply(lambda e: dict_code_nodes.get(e,-1))

- Sankey viz:

In [134]:
import plotly.graph_objects as go

# override gray link colors with 'source' colors
opacity = 0.4
# change 'magenta' to its 'rgba' value to add opacity

fig = go.Figure(data=[go.Sankey(
    valueformat = ".0f",
    valuesuffix = "Pub",
    # Define nodes
    node = dict(
      pad = 5,
      thickness = 15,
      line = dict(color = "black", width = 0.5),
      label =  list(dict_code_nodes.keys()),
      color =  colors_nodes
    ),
    # Add links
    link = dict(
      source =  df_links['source'].to_list(),
      target =  df_links['target'].to_list(),
      value =  df_links['value'].to_list()
))])

fig.update_layout(title_text="Geographic Information",
                  font_size=10)
fig.show()

In [ ]:
df_links.head(10)

,source,target,value,color
0,Africa,Northern Africa,77,"rgba(181,10,147, 0.8)"
1,Africa,Sub-Saharan Africa,532,"rgba(181,10,147, 0.8)"
2,Americas,Latin America and the Caribbean,519,"rgba(25,253,148, 0.8)"
3,Americas,Northern America,46,"rgba(25,253,148, 0.8)"
4,Asia,Central Asia,49,"rgba(69,120,231, 0.8)"
5,Asia,Eastern Asia,57,"rgba(69,120,231, 0.8)"
6,Asia,South-eastern Asia,117,"rgba(69,120,231, 0.8)"
7,Asia,Southern Asia,79,"rgba(69,120,231, 0.8)"
8,Asia,Western Asia,166,"rgba(69,120,231, 0.8)"
9,Europe,Eastern Europe,95,"rgba(1,135,9, 0.8)"


In [153]:
df_cont = pd.read_csv('../data/0_external/countries.csv',sep=',')

col_utils = ['name','region','sub-region','alpha-2']
df_cont = df_cont.loc[:,col_utils].copy()

df_cont.rename(columns={'name':'country', 'alpha-2':'country_code'}, inplace=True)

df_cont.head()

,country,region,sub-region,country_code
0,Afghanistan,Asia,Southern Asia,AF
1,Åland Islands,Europe,Northern Europe,AX
2,Albania,Europe,Southern Europe,AL
3,Algeria,Africa,Northern Africa,DZ
4,American Samoa,Oceania,Polynesia,AS


In [164]:
df_lat_long = pd.read_csv('../data/0_external/countries_lat_long.csv',sep=';', decimal='.')

df_lat_long

,country,region,sub-region,country_code,latitude,longitude
0,Afghanistan,Asia,Southern Asia,AF,33.939110,67.709953
1,Albania,Europe,Southern Europe,AL,41.153332,20.168331
2,Algeria,Africa,Northern Africa,DZ,28.033886,1.659626
3,American Samoa,Oceania,Polynesia,AS,-14.270972,-170.132217
4,Andorra,Europe,Southern Europe,AD,42.546245,1.601554
...,...,...,...,...,...,...
237,Wallis and Futuna,Oceania,Polynesia,WF,-13.768752,-177.156097
238,Western Sahara,Africa,Northern Africa,EH,24.215527,-12.885834
239,Yemen,Asia,Western Asia,YE,15.552727,48.516388
240,Zambia,Africa,Sub-Saharan Africa,ZM,-13.133897,27.849332


In [157]:
df_cont.merge(df_lat_long, how='inner', on='country_code').to_csv('countries_lat_long.csv', sep=';', decimal='.')

In [149]:
df_cont.name.nunique()

249

In [150]:
df_lat_long.country_code.nunique()

244

In [151]:
df_cont.head()

,name,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,NaN,142.0,34.0,NaN
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,NaN,150.0,154.0,NaN
2,Albania,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,NaN,150.0,39.0,NaN
3,Algeria,DZ,DZA,12,ISO 3166-2:DZ,Africa,Northern Africa,NaN,2.0,15.0,NaN
4,American Samoa,AS,ASM,16,ISO 3166-2:AS,Oceania,Polynesia,NaN,9.0,61.0,NaN


-----------